# Install Pandas

In [1]:
!pip install tensorflow scikit-learn pandas --quiet

# Import library

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, LSTM, Dense, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [6]:
df = pd.read_csv('dataSetKopi_Bersih.csv')

df['full_text'] = df[['Variety', 'Processing.Method', 'Aroma']].astype(str).agg(' '.join, axis=1)

texts = df['full_text'].astype(str)
labels = df['Species'].astype(str)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Tokenisasi teks

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
maxlen = 100
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post')

x_train1, x_test1, y_train, y_test = train_test_split(
    padded_sequences, labels_encoded, test_size=0.2, random_state=42
)

# Buat model CNN + LSTM

In [9]:
input_layer = Input(shape=(maxlen,))
embedding = tf.keras.layers.Embedding(input_dim=5000, output_dim=128, input_length=maxlen)(input_layer)

cnn = Conv1D(filters=32, kernel_size=3, activation="relu")(embedding)
cnn = GlobalMaxPooling1D()(cnn)

lstm = LSTM(32)(embedding)

concat = concatenate([cnn, lstm])


output = Dense(1, activation="sigmoid")(concat)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 128)  │    640,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 98, 32)    │     12,320 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 32)        │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │     20,608 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │         65 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 672,993 (2.57 MB)

 Trainable params: 672,993 (2.57 MB)

 Non-trainable params: 0 (0.00 B)

# Compile

In [10]:
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer='rmsprop')
history = model.fit(x_train1, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test1, y_test), verbose=1)

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.9848 - loss: 0.2196 - val_accuracy: 0.9701 - val_loss: 0.1351
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.9795 - loss: 0.1014 - val_accuracy: 0.9701 - val_loss: 0.1392
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9768 - loss: 0.1114 - val_accuracy: 0.9701 - val_loss: 0.1410
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.9779 - loss: 0.1082 - val_accuracy: 0.9701 - val_loss: 0.1354
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.9754 - loss: 0.1178 - val_accuracy: 0.9701 - val_loss: 0.1413
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.9792 - loss: 0.1026 - val_accuracy: 0.9701 - val_loss: 0.1394
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.9844 - loss: 0.0814 - val_accuracy: 0.9701 - val_loss: 0.1321
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.9817 - loss: 0.0928 - val_accuracy: 0.9701 - 

# Save Model

In [11]:
model.save("/content/drive/MyDrive/model_kopi.h5")

# Evaluasi

In [12]:
score = model.evaluate(x_test1, y_test, verbose=0)
print("Accuracy: {:.2f}%".format(score[1] * 100))

Accuracy: 97.01%
